In [19]:

# !pip install gensim
# !pip install python-Levenshtein



import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
# import nltk
# nltk.download('stopwords')
# from nltk.corpus import stopwords
from tqdm import tqdm
import re
import collections
# from wordcloud import STOPWORDS
# from scipy.sparse import csr_matrix
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
# nltk.download('vader_lexicon')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
# import string
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from wordcloud import WordCloud
import gensim
import time
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import multiprocessing
import matplotlib.image as mpimg
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LogisticRegression



# Run the next cell only if you want to train the model. 
# If already trained, Skip to the next cell and directly load the model.

In [2]:
train_data = pd.read_csv("preprocessed_tokenized_training_data.csv")
train_data['concatenated_description'] = train_data['concatenated_description'].map(str)
train_data_idf = train_data.copy()

In [3]:
train_data

In [4]:
train_data['concatenated_description'] = train_data['concatenated_description'].apply(gensim.utils.simple_preprocess)

In [5]:
cores = multiprocessing.cpu_count()

# Build the Word2Vec model
# Continous bag of words
# Train the Word2Vec model

model2 = Word2Vec(vector_size=200, window=5, min_count=1, sg=0, workers=cores-1)
model2.build_vocab(train_data['concatenated_description'], progress_per=1000)
t = time.time()
model2.train(train_data['concatenated_description'], total_examples=model2.corpus_count, epochs=20)
print('Time to build vocab for Model-2: {} mins'.format(round((time.time() - t) / 60, 2)))


In [6]:
model2.save("word2vec_cbow_model")

# One can directly load the model from here instead of training the models again.

In [7]:
#Loading a pretrained model

model2 = Word2Vec.load("word2vec_cbow_model")


In [8]:
def avg_word2vec2(doc):
    return np.mean([model2.wv[word] for word in doc if word in model2.wv.index_to_key], axis=0)

In [9]:
train_data_model2 = train_data.copy()

series = train_data_model2.concatenated_description.apply(avg_word2vec2)

df = pd.DataFrame(series.apply(pd.Series))

train_data_model2 = pd.concat([train_data_model2, df], axis=1)

train_data_model2 = train_data_model2.drop(['concatenated_description'],axis=1)

In [10]:
train_data_model2.to_csv('avgword2vec_cbow.csv')

In [20]:
train_data_model2 = pd.read_csv('avgword2vec_sg.csv')

In [21]:
train_data_model2.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'train_id'], inplace=True)

In [22]:
train_data_model2

,item_condition_id,price,shipping,0,1,2,3,4,5,6,...,190,191,192,193,194,195,196,197,198,199
0,4,6.0,1,0.037552,0.023915,0.005228,0.024870,0.125262,-0.038266,0.113200,...,0.261896,0.143941,-0.128030,0.080459,0.144991,0.017747,0.205571,-0.201369,-0.035721,-0.056952
1,1,38.0,0,0.232859,-0.038133,0.074237,0.100791,0.023038,-0.013040,0.028851,...,0.251430,0.066081,0.096789,0.106077,-0.000116,0.047183,0.072342,-0.355895,-0.056492,-0.148951
2,3,19.0,1,-0.033843,-0.001777,-0.164303,0.292715,0.050827,0.184387,0.174933,...,0.230903,0.064281,0.136571,-0.133916,0.242645,-0.154565,0.043090,-0.182490,0.015514,-0.233542
3,3,13.0,1,0.277575,-0.176075,0.103876,0.312039,-0.002630,-0.312623,0.096224,...,0.320663,0.184249,0.067134,0.063168,0.386055,0.023002,0.048363,-0.309135,0.004195,-0.173594
4,2,14.0,1,0.042444,0.112054,0.135623,0.246667,0.153029,-0.083521,0.179250,...,0.271193,0.262207,-0.010959,0.115632,-0.048844,-0.130186,0.229600,-0.207454,0.013241,-0.216210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22220,3,21.0,0,-0.074463,0.122469,0.416734,0.020856,0.094992,-0.172024,0.144668,...,0.089671,-0.003719,-0.180867,-0.000074,-0.097486,0.138730,0.137916,-0.007709,0.017316,-0.166371
22221,1,10.0,0,0.086083,-0.024563,-0.018871,0.173806,0.201046,-0.288738,-0.076850,...,0.335237,-0.205561,0.072868,-0.055953,0.042459,0.036656,0.058271,-0.196131,0.020433,-0.209770
22222,1,10.0,0,0.146066,-0.121376,0.111775,0.098236,0.063539,0.013970,-0.072577,...,0.291403,0.053669,0.114085,-0.105494,0.046369,0.136659,0.182144,-0.212911,0.126016,0.030037
22223,1,12.0,0,-0.041727,-0.142778,0.097017,0.209473,-0.015237,-0.066154,-0.294085,...,0.369271,0.002607,0.175795,-0.047760,0.027296,0.038357,0.118579,-0.114327,-0.092380,0.047048


## "avgword2vec_sg.csv" --> Will be used for "train_test_split", Contains 0.03 of entire dataset. 
## Generated from extracting dataset "preprocessed_tokenized_training_data.csv" and then applying "word2vec_cbow_model" on it.

# We've our dataset ready for the Model-2.

In [23]:
data = train_data_model2.copy()

In [24]:
from sklearn.model_selection import train_test_split

y = data['price']
X = data.drop(columns=['price'])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [25]:
X_train = X_train[:10000]
y_train = y_train[:10000]
X_test = X_test[:3500]
y_test = y_test[:3500]

In [26]:
from sklearn.decomposition import KernelPCA
kpca = KernelPCA(n_components=175)
X_K_train = kpca.fit_transform(X_train)
X_K_test = kpca.transform(X_test)

In [27]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np

param_dist = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],  # Learning rate for boosting
    'max_depth': range(3, 10),  # Maximum tree depth
    'n_estimators': [100, 200, 300, 400],  # Number of boosted trees
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],  # Subsample ratio of the training instances
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],  # Subsample ratio of columns when constructing each tree
    'gamma': [0, 0.1, 0.2, 0.3, 0.4],  # Minimum loss reduction required to make a further partition on a leaf node
    'reg_alpha': [0, 0.001, 0.01, 0.1, 1],  # L1 regularization term on weights
    'reg_lambda': [0, 0.001, 0.01, 0.1, 1],  # L2 regularization term on weights
    'min_child_weight': [1, 3, 5, 7],  # Minimum sum of instance weight needed in a child
}


xgb = XGBRegressor()

random_search = RandomizedSearchCV(estimator=xgb, param_distributions=param_dist, n_iter=15, cv=5, random_state=42, verbose=3)

random_search.fit(X_K_train, y_train)

best_params = random_search.best_params_
best_model = random_search.best_estimator_

print("Best Hyperparameters:", best_params)

y_pred = best_model.predict(X_K_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV 1/5] END colsample_bytree=0.6, gamma=0.2, learning_rate=0.01, max_depth=7, min_child_weight=7, n_estimators=400, reg_alpha=0.1, reg_lambda=0.001, subsample=0.9;, score=0.182 total time=  42.2s
[CV 2/5] END colsample_bytree=0.6, gamma=0.2, learning_rate=0.01, max_depth=7, min_child_weight=7, n_estimators=400, reg_alpha=0.1, reg_lambda=0.001, subsample=0.9;, score=0.287 total time=  42.0s
[CV 3/5] END colsample_bytree=0.6, gamma=0.2, learning_rate=0.01, max_depth=7, min_child_weight=7, n_estimators=400, reg_alpha=0.1, reg_lambda=0.001, subsample=0.9;, score=0.262 total time=  42.1s
[CV 4/5] END colsample_bytree=0.6, gamma=0.2, learning_rate=0.01, max_depth=7, min_child_weight=7, n_estimators=400, reg_alpha=0.1, reg_lambda=0.001, subsample=0.9;, score=0.163 total time=  42.4s
[CV 5/5] END colsample_bytree=0.6, gamma=0.2, learning_rate=0.01, max_depth=7, min_child_weight=7, n_estimators=400, reg_alpha=0.1, reg_lambda=0.001, s

[CV 3/5] END colsample_bytree=0.9, gamma=0.2, learning_rate=0.2, max_depth=5, min_child_weight=7, n_estimators=400, reg_alpha=0, reg_lambda=0.1, subsample=0.6;, score=0.156 total time=  34.2s
[CV 4/5] END colsample_bytree=0.9, gamma=0.2, learning_rate=0.2, max_depth=5, min_child_weight=7, n_estimators=400, reg_alpha=0, reg_lambda=0.1, subsample=0.6;, score=0.132 total time=  35.6s
[CV 5/5] END colsample_bytree=0.9, gamma=0.2, learning_rate=0.2, max_depth=5, min_child_weight=7, n_estimators=400, reg_alpha=0, reg_lambda=0.1, subsample=0.6;, score=0.077 total time=  34.7s
[CV 1/5] END colsample_bytree=1.0, gamma=0, learning_rate=0.05, max_depth=4, min_child_weight=1, n_estimators=100, reg_alpha=0.01, reg_lambda=1, subsample=1.0;, score=0.088 total time=  10.6s
[CV 2/5] END colsample_bytree=1.0, gamma=0, learning_rate=0.05, max_depth=4, min_child_weight=1, n_estimators=100, reg_alpha=0.01, reg_lambda=1, subsample=1.0;, score=0.232 total time=  10.5s
[CV 3/5] END colsample_bytree=1.0, gamma

In [28]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
import numpy as np


# Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print("Root Mean Squared Error (RMSE):", rmse)

# Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)

# R-squared (R²) or Coefficient of Determination
r2 = r2_score(y_test, y_pred)
print("R-squared (R²):", r2)

# Explained Variance Score
explained_var = explained_variance_score(y_test, y_pred)
print("Explained Variance Score:", explained_var)


Mean Squared Error (MSE): 1434.4592011139548
Root Mean Squared Error (RMSE): 37.87425512289258
Mean Absolute Error (MAE): 15.126183448655265
R-squared (R²): 0.1493341390803118
Explained Variance Score: 0.15023684282710437


In [29]:
from sklearn.model_selection import cross_val_score

num_folds = 5
scores = cross_val_score(xgb, X_K_train, y_train, cv=num_folds, scoring='neg_mean_squared_error')

# Cross-validation returns negative mean squared error, so we take the absolute values
mse_scores = -scores

# Compute the mean and standard deviation of the cross-validation scores
mean_mse = mse_scores.mean()
std_mse = mse_scores.std()
# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_mse)
# Print the results
print("Mean MSE:", mean_mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Standard Deviation of MSE:", std_mse)

Mean MSE: 1445.5082288935332
Root Mean Squared Error (RMSE): 38.01983993776845
Standard Deviation of MSE: 367.0868743231442


In [30]:
import pickle
filename = 'sg_model6.pkl'
with open(filename, 'wb') as file:
    pickle.dump(best_model, file)
